# End to End MLPerf Submission

This is following the [General MLPerf Submission Rules](https://github.com/mlcommons/policies/blob/master/submission_rules.adoc).


### Change working Directory to official MLCommons Inference Repo to install loadgen

In [ ]:
%cd ../inference

### Build loadgen

In [ ]:
# build loadgen
!pip install pybind11
!cd loadgen; CFLAGS="-std=c++14 -O3" python setup.py develop

In [ ]:
!cd vision/classification_and_detection; python setup.py develop

### Set Working Directory

In [ ]:
%cd ../Python

# Download data

For this example, the ImageNet and/or COCO validation data should already be on the host system. See the [MLPerf Image Classification task](https://github.com/mlcommons/inference/tree/master/vision/classification_and_detection#datasets) for more details on obtaining this. For the following step each validation dataset is stored in /workspace/data/. You should change this to the location in your setup.

In [ ]:
%%bash

mkdir data
ln -s /data/imagenet2012 data/
ln -s /data/coco data/

### Download models

In [ ]:
%%bash

mkdir models

# resnet50
wget -q https://zenodo.org/record/2535873/files/resnet50_v1.pb -O models/resnet50_v1.pb 
wget -q https://zenodo.org/record/2592612/files/resnet50_v1.onnx -O models/resnet50_v1.onnx

# ssd-mobilenet
wget -q http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz -O models/ssd_mobilenet_v1_coco_2018_01_28.tar.gz
tar zxvf ./models/ssd_mobilenet_v1_coco_2018_01_28.tar.gz -C ./models; mv models/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb ./models/ssd_mobilenet_v1_coco_2018_01_28.pb
wget -q https://zenodo.org/record/3163026/files/ssd_mobilenet_v1_coco_2018_01_28.onnx -O models/ssd_mobilenet_v1_coco_2018_01_28.onnx 

# ssd-resnet34
wget -q https://zenodo.org/record/3345892/files/tf_ssd_resnet34_22.1.zip -O models/tf_ssd_resnet34_22.1.zip
unzip ./models/tf_ssd_resnet34_22.1.zip -d ./models; mv models/tf_ssd_resnet34_22.1/resnet34_tf.22.1.pb ./models
wget -q https://zenodo.org/record/3228411/files/resnet34-ssd1200.onnx -O models/resnet34-ssd1200.onnx

### Run benchmarks using MLModelScope as SUT

Lets prepare a submission for ResNet-50 on a MLModelScope machine with a NVIDIA RTX3090 GPU using PyTorch, MXNet, OnnxRuntime. 

The following script will run those combinations and prepare a submission directory, following the general submission rules documented [here](https://github.com/mlperf/policies/blob/master/submission_rules.adoc).

>benchmark must be one of {resnet50, ssd-mobilenet, ssd-resnet34, rnnt, bert-99, bert-99.9, dlrm-99, dlrm-99.9, 3d-unet-99, 3d-unet-99.9}.  

**But we used our model name in MLModelScope instead**

In [ ]:
import logging
import os
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# final results go here
ORG = "MLModelScope"
DIVISION = "open"
SUBMISSION_ROOT = "/tmp/mlperf-submission"
SUBMISSION_DIR = os.path.join(SUBMISSION_ROOT, DIVISION, ORG)
os.environ['SUBMISSION_ROOT'] = SUBMISSION_ROOT
os.environ['SUBMISSION_DIR'] = SUBMISSION_DIR
os.makedirs(SUBMISSION_DIR, exist_ok=True)
os.makedirs(os.path.join(SUBMISSION_DIR, "measurements"), exist_ok=True)
os.makedirs(os.path.join(SUBMISSION_DIR, "code"), exist_ok=True)

In [ ]:
%%bash

# options for official runs
gopt="--samples-per-query 40 --qps 100"

function one_run {
    # args: mode count framework device model ...
    scenario=$1; shift
    count=$1; shift
    framework=$1
    device=$2
    model_name=$3
    dataset=$4
    max_batchsize=$5
    system_id=$framework-$device
    echo "====== $model_name/$scenario ====="

    output_dir=$SUBMISSION_DIR/results/$system_id/$model_name
    
    use_gpu=0

    if [ $device == "cpu" ]  
    then
        use_gpu=0
    else
        use_gpu=1
    fi

    # accuracy run
    
    mkdir -p $output_dir/$scenario/accuracy

    # python3 main.py --dataset $dataset --scenario $scenario --max-batchsize $max_batchsize --backend $framework --model-name $model_name --accuracy --use-gpu $use_gpu --log_dir $output_dir/$scenario/accuracy $gopt \
    #     > $output_dir/$scenario/accuracy/accuracy.txt
    # cat $output_dir/$scenario/accuracy/accuracy.txt

    # performance run
    cnt=0
    while [ $cnt -lt $count ]; do
        let cnt=cnt+1
        python3 main.py --dataset $dataset --scenario $scenario --max-batchsize $max_batchsize --backend $framework --model-name $model_name --use-gpu $use_gpu --log_dir $output_dir/$scenario/performance/run_$cnt $gopt
    done
    
    # setup the measurements directory
    mdir=$SUBMISSION_DIR/measurements/$system_id/$model_name/$scenario
    mkdir -p $mdir
    cp ../inference/mlperf.conf $mdir

    # reference app uses command line instead of user.conf
    echo "# empty" > $mdir/user.conf
    touch $mdir/README.md
    impid="demo"
    cat > $mdir/$system_id"_"$impid"_"$scenario".json" <<EOF
    {
        "input_data_types": "fp32",
        "retraining": "none",
        "starting_weights_filename": "https://zenodo.org/record/2592612/files/resnet50_v1.onnx",
        "weight_data_types": "fp32",
        "weight_transformations": "none"
    }
EOF
}

function one_model {
    # args: framework device model ...
    one_run SingleStream 1 $@ --max-latency 0.0005
    # one_run Server 1 $@
    one_run Offline 1 $@ --qps 1000
    # one_run MultiStream 1 $@
}


# run image classifier benchmarks 
export DATA_DIR=/data/imagenet2012linear
# export DATA_DIR=$DATA_ROOT/coco

# one_model <framework> <gpu/cpu> <model-name> <dataset> <max-batchsize>
# one_model onnxruntime gpu MLPerf_ResNet50_v1.5 imagenet 64
# one_model onnxruntime gpu TorchVision_VGG_11 imagenet 64
one_model onnxruntime gpu TorchVision_VGG_11 imagenet 64
one_model onnxruntime gpu TorchVision_ResNet_18 imagenet 64
one_model onnxruntime gpu TorchVision_ResNet_34 imagenet 64
one_model onnxruntime gpu TorchVision_ResNet_101 imagenet 64
one_model onnxruntime gpu TorchVision_ResNet_152 imagenet 64
# one_model onnxruntime gpu resnet50 coco 1 $gopt

There might be large trace files in the submission directory - we can delete them.

In [ ]:
!find {SUBMISSION_DIR}/ -name mlperf_log_trace.json -delete

### Complete submission directory

Add the required meta data to the submission.

In [ ]:
%%bash

#
# setup systems directory
#
if [ ! -d ${SUBMISSION_DIR}/systems ]; then
    mkdir ${SUBMISSION_DIR}/systems
fi

cat > ${SUBMISSION_DIR}/systems/onnxruntime-gpu.json <<EOF
{
        "division": "open",
        "status": "available",
        "submitter": "mlmodelscope",
        "system_name": "onnxruntime-gpu",
        "system_type": "datacenter",
        
        "number_of_nodes": 1,
        "host_memory_capacity": "32GB",
        "host_processor_core_count": 16,
        "host_processor_frequency": "3.60GHz",
        "host_processor_model_name": "Intel(R) Core(TM) i7-7820X CPU @ 3.60GHz",
        "host_processors_per_node": 1,
        "host_storage_capacity": "2TB",
        "host_storage_type": "Hard Drive",
        
        "accelerator_frequency": "-",
        "accelerator_host_interconnect": "-",
        "accelerator_interconnect": "-",
        "accelerator_interconnect_topology": "-",
        "accelerator_memory_capacity": "12GB",
        "accelerator_memory_configuration": "none",
        "accelerator_model_name": "TITAN V",
        "accelerator_on-chip_memories": "-",
        "accelerators_per_node": 1,

        "framework": "v1.6.0",
        "operating_system": "ubuntu-18.04",
        "other_software_stack": "cuda-10.2",
        "sw_notes": ""
}
EOF

cat > ${SUBMISSION_DIR}/systems/pytorch-gpu.json <<EOF
{
        "division": "open",
        "status": "available",
        "submitter": "mlmodelscope",
        "system_name": "pytorch-gpu",
        "system_type": "datacenter",
        
        "number_of_nodes": 1,
        "host_memory_capacity": "32GB",
        "host_processor_core_count": 16,
        "host_processor_frequency": "3.60GHz",
        "host_processor_model_name": "Intel(R) Core(TM) i7-7820X CPU @ 3.60GHz",
        "host_processors_per_node": 1,
        "host_storage_capacity": "2TB",
        "host_storage_type": "Hard Drive",
        
        "accelerator_frequency": "-",
        "accelerator_host_interconnect": "-",
        "accelerator_interconnect": "-",
        "accelerator_interconnect_topology": "-",
        "accelerator_memory_capacity": "12GB",
        "accelerator_memory_configuration": "none",
        "accelerator_model_name": "TITAN V",
        "accelerator_on-chip_memories": "-",
        "accelerators_per_node": 1,

        "framework": "v1.5.0",
        "operating_system": "ubuntu-18.04",
        "other_software_stack": "cuda-10.2",
        "sw_notes": ""
}
EOF

In [ ]:
%%bash

#
# setup code directory
#
dir=${SUBMISSION_DIR}/code/resnet/reference
mkdir -p $dir
echo "git clone https://github.com/yhchang3/mlcommons-mlmodelscope.git" > $dir/VERSION.txt
git rev-parse HEAD >> $dir/VERSION.txt

### What's in the submission directory now ?


In [ ]:
!find {SUBMISSION_ROOT}/ -type f

## Run the submission checker

Finally, run the submission checker tool that does some sanity checking on your submission.
We run it at the end and attach the output to the submission.

In [ ]:
!python ../inference/tools/submission/submission-checker.py --input {SUBMISSION_ROOT} > {SUBMISSION_DIR}/submission-checker.log 2>&1 
!cat {SUBMISSION_DIR}/submission-checker.log